Interesting LDA article: https://medium.com/data-science/practical-guide-to-topic-modeling-with-lda-05cd6b027bdf

Step-by-step: https://www.google.com/url?q=https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/&sa=D&source=docs&ust=1755572810705545&usg=AOvVaw3qRIvRnelPAxkHEFeDfU7F


In [2]:
import warnings

warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning)

In [ ]:
!pip install gensim
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompati

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.9 MB/s eta 0:00:00
^C


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 29 13:54:48 2025

@author: dnd2129
NLP processes
"""

import pandas as pd

import numpy as np

from google.colab import drive

import re

import spacy

from scipy.stats import uniform

from pprint import pprint

import gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import KMeans




#nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download("omw-1.4")
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer


#Plots
#import pyLDAvis
#import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


now, combine all courses for each student into one row so we have one row per studnt. Then,tokenize it like I did above (choose uni/bi or trigrams depending of which worked best).
THEN: only retain the tokens that I manually chose to use from the manual mapping.

# Grouping by student
One row per student-- adding a course_list column which contains all courses they have taken.

## Main DF

In [4]:
# mount gdrive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.read_excel("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/raw data/Project - Student Course Clustering/Student Course Elective Enrollments Graduates 2016-2025.xlsx")
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined Native Flag,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name
0,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services
1,5434,C002941648,aa2301,2016,20153MGMTB8515001,Elective,MGMTB8515,MANAGEMENT,3.0,HP,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Top Management Process
2,9020,C002941648,aa2301,2016,20151MGMTB8538007,Elective,MGMTB8538,MANAGEMENT,1.5,H,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,The Leader's Voice: Communication Skills for L...
3,19900,C002941648,aa2301,2016,20151MGMTB8536001,Elective,MGMTB8536,MANAGEMENT,1.5,HP,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Strategy and Competition in Pharmaceuticals an...
4,41190,C002941648,aa2301,2016,20153MGMTB8507002,Elective,MGMTB8507,MANAGEMENT,1.5,H-,...,Not American Indian or Alaska Native,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Organizational Change


### Cleaning Text
Cleaning combined name same way I did to obtain the yake keywords (in order to map exactly)

In [6]:

#cleaning texts
df['clean_name'] = df['combined_name'].fillna('') #changing nan to blank
df['clean_name'] = df['clean_name'].str.replace(r'\s+', ' ', regex=True)#cleaning new line chars
df['clean_name'] = df['clean_name'].str.replace("'", '', regex=False)#removing single quotes

df.head()


,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
0,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming the Delivery of Healthcare Services
1,5434,C002941648,aa2301,2016,20153MGMTB8515001,Elective,MGMTB8515,MANAGEMENT,3.0,HP,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Top Management Process,Top Management Process
2,9020,C002941648,aa2301,2016,20151MGMTB8538007,Elective,MGMTB8538,MANAGEMENT,1.5,H,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,The Leader's Voice: Communication Skills for L...,The Leaders Voice: Communication Skills for Le...
3,19900,C002941648,aa2301,2016,20151MGMTB8536001,Elective,MGMTB8536,MANAGEMENT,1.5,HP,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Strategy and Competition in Pharmaceuticals an...,Strategy and Competition in Pharmaceuticals an...
4,41190,C002941648,aa2301,2016,20153MGMTB8507002,Elective,MGMTB8507,MANAGEMENT,1.5,H-,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Organizational Change,Organizational Change


#### Remove stopwords

In [7]:
def rem_sw(var_in):
    sw = list(set(stopwords.words('english')))
    tmp = [word for word in var_in.split() if word not in sw]
    return ' '.join(tmp)

In [8]:
df['clean_name'] = df['clean_name'].apply(rem_sw)
df.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,Combined NHOPI Flag,Combined White Flag,Combined Federal Race,Slate Citizenship 2,Slate Citizenship 2.1,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name
0,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services
1,5434,C002941648,aa2301,2016,20153MGMTB8515001,Elective,MGMTB8515,MANAGEMENT,3.0,HP,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Top Management Process,Top Management Process
2,9020,C002941648,aa2301,2016,20151MGMTB8538007,Elective,MGMTB8538,MANAGEMENT,1.5,H,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,The Leader's Voice: Communication Skills for L...,The Leaders Voice: Communication Skills Leadin...
3,19900,C002941648,aa2301,2016,20151MGMTB8536001,Elective,MGMTB8536,MANAGEMENT,1.5,HP,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Strategy and Competition in Pharmaceuticals an...,Strategy Competition Pharmaceuticals Biotechno...
4,41190,C002941648,aa2301,2016,20153MGMTB8507002,Elective,MGMTB8507,MANAGEMENT,1.5,H-,...,Not Native Hawaiian or Other Pacific Islander,White,Two or More,United States,NaN,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Organizational Change,Organizational Change


# Title LDA

### Checking if any student doesnt have a course

In [ ]:
no_course_mask = df["clean_name"].apply(lambda ks: len(ks) == 0)
n_no_crs = no_course_mask.sum()
n_total = len(df)

print(f"{n_no_crs} out of {n_total} students have no course_name.")


## Importing the combined name and yake map

In [ ]:
keyword_yake5_map = pd.read_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/keyword-yake5-map.csv")
keyword_yake5_map.head()

In [ ]:
#Split the string into a list of keywords
keyword_yake5_map["keyword_list"] = keyword_yake5_map["yake_top5_keywords"].apply(
    lambda x: [kw.strip() for kw in re.split(r"[;,]", str(x)) if kw.strip()]
)

#Explode so each keyword gets its own row
keyword_long = keyword_yake5_map.explode("keyword_list", ignore_index=True)

#Renaming col
keyword_long = keyword_long.rename(columns={"keyword_list": "keyword"})

keyword_long.head(10)

### Keep MAP

Importing the manually coded csv which has that keywords to keep (keep map)

In [ ]:
keep_map = pd.read_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/keywords_keep_map.csv")
keep_map.head()

#### keep tagged keywords and drop the rest

merging with full keyword list with keep map to mark which to keep, along with lsit of keywords

In [ ]:
tagged_keywords = keyword_long.merge(keep_map, left_on='keyword', right_on = "unique yake_top5_keywords", how='left')
tagged_keywords.head()


NOT filtering for kept keywords only yet as I want a fallback for coursenames which might not have any keywords after filtering out 0s in the keep tag. So, I want to lemmatize all keywords and retain it until the end just in case I can uses keep tag 0s as fallback for courses with no tags.

### Stemming and Lemmetize the keywords (Checking)

Adding them to the same dataframe as separate cols to test which might work best. I think lemmas should work best as it makes sure the tokens are actual words, unlike stemming.

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

tagged_keywords["stem"] = tagged_keywords["keyword"].apply(
    lambda x: stemmer.stem(x))

In [ ]:
nlp = spacy.load("en_core_web_sm")
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

tagged_keywords['lemma'] = tagged_keywords['keyword'].apply(lambda x: lemmatize_text(x))
tagged_keywords.head()


After running with normal lemmatizer, I noticed some discrapencies which I will manually override (below) to make sure certain tokens converge

In [ ]:
lemm = WordNetLemmatizer()

# Explicit fixes for tricky derivations
OVERRIDES = {
    "financial": "finance",
    "finance": "finance",
    "finances":"finance",
    "analytics": "analysis",
    "analysis": "analysis",
    "accounting": "accounting",
    "accounts":"accounting",
    "accounting":"accounting",
    "family" : "family",
    "art":"art",
    "arts":"art",
    "invest": "investing",
    "investing":"investing",
    "investment":"investing",
    "entrepreneurship": "entrepreneurship",
    "entrepreneurial": "entrepreneurship",
    "entrepreneur": "entrepreneurship",
    "tech":"technology",
    "technology": "technology",
    "technological":"technology",
    "technologies":"technology",
    "digital":"technology",
    "risk":"rise",
    "science":"science",
    "sciences":"science",
    "scientist":"science",
    "consulting":"consulting",
    "consultant":"consulting",
    "consult":"consulting",
    "consultancy": "consulting",
    "pharma":"pharmaceutical",
    "pharmaceutical":"pharmaceutical",
    "pharmaceuticals":"pharmaceutical",
    "health":"healthcare",
    "healthcare":"healthcare",
    "retailing":"retail",
    "retail":"retail",
    "leader":"leadership",
    "leadership":"leadership",
    "strategy":"strategy",
    "strategies":"strategy",
    "strategic":"strategy",
    "products":"product",
    "security":"securities",
    "securities":"securities",
    "macroeconomic":"macroeconomic",
    "macroeconomy":"macroeconomic",
    "start-up":"startup",
    "startup":"startup",
    "securities analysis":"security analysis",
    "economy":"economy",
    "economic":"economy",
    "economics":"economy",
    "activist":"activism",
    "activism":"activism",
    "sale":"sales",
    "sales":"sales",
    "big":"big data",
    "estate":"real estate",
    "real":"real estate",
    "sustainable":"sustainability",
    "medium":"",
    "program":"",
    "rise":"",
}

def get_root_one(word: str) -> str:
    if not isinstance(word, str) or not word.strip():
        return word
    w = word.strip().lower()
    if w in OVERRIDES:
        return OVERRIDES[w]

    candidates = []
    for ss in wn.synsets(w):
        for lem in ss.lemmas():
            for rel in lem.derivationally_related_forms():
                form = rel.name().lower().replace("_", " ")
                pos  = rel.synset().pos()   # 'n','v','a','r','s'
                freq = rel.count()          # WordNet frequency
                candidates.append((form, pos, freq))

    # add fallbacks (original + simple lemmas)
    candidates.append((w, 'x', 0))
    candidates.append((lemm.lemmatize(w, 'n'), 'n', 0))
    candidates.append((lemm.lemmatize(w, 'v'), 'v', 0))
    candidates.append((lemm.lemmatize(w, 'a'), 'a', 0))

    # pick best: prefer nouns, then higher frequency, then shorter string
    form = sorted(candidates, key=lambda t: (t[1] != 'n', -t[2], len(t[0]), t[0]))[0][0]
    return form

# Apply (returns a single token per keyword)
tagged_keywords["lemma_root"] = tagged_keywords["keyword"].apply(get_root_one)
tagged_keywords.head()


lemma_root has the keywords I want (lemma has some deviations and doesn't have the manual override).

### Fitlering only for keywords we want to keep (keep == 1)

In [ ]:
kept_keywords = tagged_keywords[tagged_keywords["keep tag"] == 1]
kept_keywords.head()

In [ ]:
kept_keywords.tail()

In [ ]:
kept_keywords.to_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/stem_lemmatized_kept_keywords.csv")

#### Looks like lemma_root might be best to use for LDA model training!!! :) buuuut-- need to merge with df on clean_name first.

## Merging df with keywords

In [ ]:
df_keywords = df.merge(
    kept_keywords,
    left_on='clean_name',
    right_on="combined_name",
    how='left'   # <-- keep ALL courses
)

df_keywords.head()

### Some courses don't have keywords--- this is a problem as we will miss those courses in our LDA.

In [ ]:
#count NAs
df_keywords["lemma_root"].isna().sum()

for courses with no keywords, I'm going to get their Top 1 keyword using YAKE on combined_name_x , then get lemma_root their yaked_top_1 again as these courses don't seem to be picked up in the keyword_long df itself- courses with nas with missing in the df_keywords were merged out because they don't exist in "kept_keywords" df, which means they don't exist in the "keywords_long" df.


In [ ]:
!pip install yake

In [ ]:
from yake import KeywordExtractor
kw_extractor = KeywordExtractor(lan="en", n=1, top=5)

def yake_top_keywords(text):
    if pd.isna(text) or not str(text).strip():
        return None
    try:
        keywords = kw_extractor.extract_keywords(text)
        return [kw for kw, score in keywords]
    except:
        return None

df_keywords["yake_top5_keywords"] = df_keywords.apply(
    lambda row: row["keyword"] if pd.notna(row["keyword"]) else yake_top_keywords(row["combined_name_x"]),
    axis=1
)

df_keywords.head()

Now, I will use a function that goes through all the top 5 yaked keywords for these courses with Na in "keyword" column-- For each row,  I will look inside the "yake_top5_keywords" for keywords which have 1 in the keep map. where it does, that will be the "keyword". if there isn't one, they I will just pick th first keyword in "yake_top5_keyword". Then, I will apply the get_root_one function for these keywords only. the lemma root col will then be populated for those which had NA.

In [ ]:
import pandas as pd

# Build keep set for quick lookup
keep_set = set(tagged_keywords.loc[tagged_keywords["keep tag"] == 1,
                                   "unique yake_top5_keywords"].str.lower())

def choose_keyword(row):
    if pd.notna(row["keyword"]):  # already has keyword, do nothing
        return row["keyword"]

    # candidates is already a list of YAKE keywords
    candidates = row["yake_top5_keywords"]

    if not isinstance(candidates, list) or not candidates:
        return None

    # check if any candidate is in keep_set
    for c in candidates:
        if c.lower() in keep_set:
            return c  # first valid keep keyword

    # otherwise fallback to first YAKE keyword
    return candidates[0]


df_keywords_final = df_keywords.copy(deep=True)
# Fill missing keywords
df_keywords_final["keyword"] = df_keywords.apply(choose_keyword, axis=1)

# Apply lemmatization only where keyword was filled from NA
def fill_root(row):
    if pd.isna(row.get("lemma_root")) and pd.notna(row["keyword"]):
        return get_root_one(row["keyword"])
    return row.get("lemma_root")

df_keywords_final["lemma_root"] = df_keywords_final.apply(fill_root, axis=1)


In [ ]:
df_keywords_final.head()

In [ ]:
#count NAs
df_keywords_final["lemma_root"].isna().sum() #there are now only 7701 without lemma_root-- this is the same number of rows which didn't have a coursename to begin with.

## Grouping by Uni

## Getting one row per student, courname list and keyword list

In [ ]:
# Group by student (all courses + all lemma_root kept keywords) ---# regroup per student, keeping both course list and keyword list
# Pick the correct course and keyword column names that actually exist--- going with lemma_root!
# One row per student: list of all courses + list of lemma-root kept keywords (as-is)
df_student = (
    df_keywords_final.groupby("uni", dropna=False)
    .agg(
        courses=("combined_name_x", list),
        keywords=("lemma_root",  list)
    )
    .reset_index()
)

df_student.head()





Removing nan and empty strings from the keywords list column

In [ ]:
df_student["keywords"] = df_student["keywords"].apply(
    lambda lst: [str(k).strip() for k in lst if pd.notna(k) and str(k).strip()]
)

df_student.head()

## LDA

### Build the “corpus” for LDA

### Checking if any student doesnt have a keyword

In [ ]:
# Count how many students have empty keyword lists
no_kw_mask = df_student["keywords"].apply(lambda ks: len(ks) == 0)
n_no_kw = no_kw_mask.sum()
n_total = len(df_student)

print(f"{n_no_kw} out of {n_total} students have no keywords.")


## Vectorize the documents (tokens)-- to get document-ter matrix

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)  # because its already preprocessed
X = vectorizer.fit_transform(df_student["keywords"])

Visualize X

In [ ]:
# Get feature/keyword names
feature_names = vectorizer.get_feature_names_out()

# Convert a small slice to dense for inspection
sample = X[:5].toarray()   # first 5 students

df_preview = pd.DataFrame(sample, columns=feature_names)
print(df_preview.head())

## Keyword correlation matrix

In [ ]:
import pandas as pd

terms = vectorizer.get_feature_names_out()
X_df = pd.DataFrame(X.toarray(), columns=terms)

correlation_matrix = X_df.corr()
correlation_matrix.head()


In [ ]:
correlation_matrix.to_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/Student Cluster Analysis GDrive/keyword correlation_matrix.csv")

## Check Matrix Sparsicity

In [ ]:
'''
Check the Sparsicity
'''
# number of non-zero entries
nnz = X.nnz

# total number of entries
total = X.shape[0] * X.shape[1]

# sparsity (proportion of zero entries)
sparsity = 1 - (nnz / total)

print(f"Shape: {X.shape}")
print(f"Non-zeros: {nnz}")
print(f"Sparsity: {sparsity:.4f} ({sparsity*100:.2f}% zeros)")


Checking an LDA model with arbitrary number of components

LDA scoring parameters:
* **Log-likelihood:** it measures how well the current model explains the
observed data. In general, this quantity has a negative range and increasing values indicate a better model fit. The log likelihood of a model can be used to monitor training progress and model convergence.
 * However, this measure is directly influenced by the number of tokens in the corpus. Therefore, perplexity usually is a better choice as it is normalized by the number of tokens and measures the per-word log likelihood.
* **Perplexity:** In contrast, perplexity has a positive value range and decreasing values indicate a better model fit.
  * Perplexity = e^(-log likelihood/number of tokens)

In [ ]:
'''
Build LDA model with sklearn
'''

lda_model = LatentDirichletAllocation(n_components=5,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',
                                      random_state=42,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(X)
print(lda_model)  # Model attributes

Checking model performance overall

In [ ]:
'''
Diagnose model performance with perplexity and log-likelihood

'''
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(X))
#Log Likelihood:  -2141711.0363983824
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(X))
#Perplexity:  133.36523412184394
# See model parameters
pprint(lda_model.get_params())

## RandomSearch-- just to quickly get an idea--
However, we need to use GridSearch to be sure.

In [ ]:

# Define the parameter distributions
param_dist = {
# Init the Model
    'n_components': [1,2,3,4,5,6,7,8,9,10,15,20,25,30],              # Number of topics
    'learning_decay': [.5,.7],        #according to article, learning decay SHOULD be >.5, and around .7 is optimal.
    'batch_size': [128,256,2000],
}

# Initialize the LDA model
lda = LatentDirichletAllocation(learning_method='online', random_state=42)

# Initialize Randomized Search
random_search = RandomizedSearchCV(
    estimator=lda,
    param_distributions=param_dist,
    n_iter=10,                       # Number of parameter combinations to try
    scoring=None,         # Or another relevant metric
    cv=3,
    verbose=1,
    random_state=42
)

# Fit to your vectorized data
random_search.fit(X)

# Best model and its parameters
print("Best Model's Params:", random_search.best_params_)
print("Best Score:", random_search.best_score_)



In [ ]:
best_lda_model = random_search.best_estimator_

## GridSearch
for the best model for n_components (number of topics) and learning decay

In [ ]:
'''
GridSearch the best LDA model
'''
# Define Search Param
search_params = {'n_components': [1,2,3,4,5,6,7,8,9,10],
                 'learning_decay': [.5,.7], #according to article, learning decay SHOULD be >.5, and around .7 is optimal.
                 'batch_size': [128, 256, 2000]}
# Init the Model
lda = LatentDirichletAllocation()
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params, n_jobs=-1) #n_jobs=-1 -->parallelizing the process

# Do the Grid Search
model.fit(X)

'''
Best model and Best parameters
'''
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(X))


## Plotting Performance scores
(log_likelihood) to see best parameters

In [ ]:

# Use cv_results_ (not best_params_)
cv = random_search.cv_results_

# Extract arrays
ld = cv["param_learning_decay"].astype(float)   # learning_decay per run
nc = cv["param_n_components"].astype(int)       # number of topics per run
scores = np.asarray(cv["mean_test_score"], dtype=float)

def series_for_decay(decay):
    m = (ld == decay)
    t = nc[m]
    s = scores[m]
    order = np.argsort(t)
    return t[order], s[order]

plt.figure(figsize=(8,5))

# plot for decay=0.5 if present
if np.any(ld == 0.5):
    t5, s5 = series_for_decay(0.5)
    plt.plot(t5, s5, marker='o', label='learning_decay = 0.5')

# plot for decay=0.7 if present
if np.any(ld == 0.7):
    t7, s7 = series_for_decay(0.7)
    plt.plot(t7, s7, marker='s', label='learning_decay = 0.7')

plt.title("Choosing Optimal LDA Model")
plt.xlabel("Number of Topics (n_components)")
plt.ylabel("Mean CV Score (log-likelihood)")
plt.grid(True)
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


In [ ]:
print("Topic numbers for decay 0.9:", sorted(set(t5)))
print("Topic numbers for decay 0.7:", sorted(set(t7)))


## Document-Topic Matrix

In [ ]:
# Document-topic distribution
lda_output = best_lda_model.transform(X)

# Column names
topicnames = [f"Topic{i}" for i in range(best_lda_model.n_components)]

# Use student IDs (or keep Doc0..DocN if you prefer)
docnames = df_student["uni"].tolist()

# Build DataFrame
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Dominant topic + prob
df_document_topic["dominant_topic"] = df_document_topic[topicnames].values.argmax(axis=1)
df_document_topic["dominant_prob"]  = df_document_topic[topicnames].max(axis=1)

# Styling
def color_green(val):
    return "color: green" if val > 0.1 else "color: black"
def make_bold(val):
    return "font-weight: 700" if val > 0.1 else "font-weight: 400"

styled_preview = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)

styled_preview


In [ ]:
'''
Review topics distribution across documents
'''
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution.head(10)

## visualize the LDA model with pyLDAvis-- broken

In [ ]:

import pyLDAvis

# 1. Topic–term distributions
topic_term_dists = best_lda_model.components_
topic_term_dists = topic_term_dists / topic_term_dists.sum(axis=1, keepdims=True)

# 2. Document–topic distributions
doc_topic_dists = best_lda_model.transform(X)

# 3. Vocabulary
vocab = vectorizer.get_feature_names_out()

# 4. Document lengths
doc_lengths = np.asarray(X.sum(axis=1)).ravel()

# 5. Term frequencies
term_frequency = np.asarray(X.sum(axis=0)).ravel()

# Check
print(topic_term_dists.shape, doc_topic_dists.shape, len(vocab), len(doc_lengths), len(term_frequency))



In [ ]:

pyLDAvis.enable_notebook()
panel = pyLDAvis.prepare(best_lda_model, X, vectorizer, mds='tsne', vocab= vocab, term_frequency=term_frequency)
panel

## Topic Keywords

In [ ]:
'''
see the Topic’s keywords
'''

# Column names = vocabulary
terms = vectorizer.get_feature_names_out()            # shape: (n_terms,)

# Row names = topics
topicnames = [f"Topic{i}" for i in range(best_lda_model.n_components)]

# Topic–keyword matrix as a DataFrame
df_topic_keywords = pd.DataFrame(best_lda_model.components_,  # (n_topics, n_terms)
                                 index=topicnames,
                                 columns=terms)

# View top-N keywords per topic
TOP_N = 10
for t in range(best_lda_model.n_components):
    top_idx = np.argsort(df_topic_keywords.iloc[t].values)[-TOP_N:][::-1]
    top_terms = terms[top_idx]
    top_weights = df_topic_keywords.iloc[t, top_idx].values
    print(f"Topic {t}: " + ", ".join(top_terms))


### Forcing keyword excusivity per topic

In [ ]:
import numpy as np

# Components: (n_topics, n_terms)
phi = best_lda_model.components_            # learned topic-word weights
n_topics, n_terms = phi.shape

# For each term, pick the topic with the max weight
word_to_topic = phi.argmax(axis=0)          # shape: (n_terms,)

# Build a term→topic one-hot matrix W: (n_terms, n_topics)
W = np.zeros((n_terms, n_topics), dtype=np.float32)
W[np.arange(n_terms), word_to_topic] = 1.0

# Document-term matrix X: (n_docs, n_terms)
# Compute hard topic counts per document: (n_docs, n_topics)
T_counts = X @ W

# Normalize to get per-doc topic proportions (document–topic matrix)
T_sums = T_counts.sum(axis=1, keepdims=True)
T_sums[T_sums == 0] = 1.0                   # avoid divide-by-zero
theta_hard = T_counts / T_sums              # (n_docs, n_topics)

# Optional: top words per hard topic
terms = vectorizer.get_feature_names_out()
topn = 25
for t in range(n_topics):
    idx = np.where(word_to_topic == t)[0]
    # Rank those words by original phi weight for topic t
    if len(idx):
        order = np.argsort(phi[t, idx])[::-1][:topn]
        print(f"Topic {t}:", ", ".join(terms[idx[order]]))
    else:
        print(f"Topic {t}: (no words assigned)")


## cluster plot

In [ ]:
'''
cluster documents that share similar topics and plot
'''
# Construct the k-means clusters

clusters = KMeans(n_clusters=15, random_state=100).fit_predict(lda_output)
# Build the Singular Value Decomposition(SVD) model
svd_model = TruncatedSVD(n_components=2)  # 2 components
lda_output_svd = svd_model.fit_transform(lda_output)
# X and Y axes of the plot using SVD decomposition
x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]
# Weights for the 15 columns of lda_output, for each component
print("Component's weights: \n", np.round(svd_model.components_, 2))
# Percentage of total information in 'lda_output' explained by the two components
print("Perc of Variance Explained: \n", np.round(svd_model.explained_variance_ratio_, 2))


In [ ]:
'''
Plot clusters
'''
# Plot
plt.figure(figsize=(12, 12))
plt.scatter(x, y, c=clusters)
plt.xlabel('Component 2')
plt.xlabel('Component 1')
plt.title("Segregation of Topic Clusters", )

# LDA prep for Description

## Merging df with course Descriptions

In [9]:
desc = pd.read_csv("/content/drive/MyDrive/CBS/CBS-Github-DND/Student Cluster Analysis and LDA-20250714T173604Z-1-001/Student Cluster Analysis and LDA/raw data/Project - Student Course Clustering/Up to 20213 Course Descriptions.csv")
desc.head()

,yearTerm,course_number,section,course_title,course_description
0,20213.0,B8444,1,Modern Value,"You might have heard that value, quant value t..."
1,20203.0,B8467,1,The Economics and Politics of Digital Disruption,You canâ€™t disrupt any industry without dislo...
2,20213.0,B8467,1,The Economics and Politics of Digital Disruption,You can&rsquo;t disrupt any industry without d...
3,20181.0,B9651,1,MS Marketing Analytics,You are a manager in a large corporation taske...
4,20163.0,B8617,1,Marketing Research and Analytics,You are a manager in a large corporation taske...


In [10]:
desc["clean_name"] = desc["course_title"].fillna('') #changing nan to blank
desc['clean_name'] = desc['clean_name'].str.replace(r'\s+', ' ', regex=True)#cleaning new line chars
desc['clean_name'] = desc['clean_name'].str.replace("'", '', regex=False)#removing single quotes

desc.head()

,yearTerm,course_number,section,course_title,course_description,clean_name
0,20213.0,B8444,1,Modern Value,"You might have heard that value, quant value t...",Modern Value
1,20203.0,B8467,1,The Economics and Politics of Digital Disruption,You canâ€™t disrupt any industry without dislo...,The Economics and Politics of Digital Disruption
2,20213.0,B8467,1,The Economics and Politics of Digital Disruption,You can&rsquo;t disrupt any industry without d...,The Economics and Politics of Digital Disruption
3,20181.0,B9651,1,MS Marketing Analytics,You are a manager in a large corporation taske...,MS Marketing Analytics
4,20163.0,B8617,1,Marketing Research and Analytics,You are a manager in a large corporation taske...,Marketing Research and Analytics


In [11]:
desc['clean_name'] = desc['clean_name'].apply(rem_sw)
desc.head()

,yearTerm,course_number,section,course_title,course_description,clean_name
0,20213.0,B8444,1,Modern Value,"You might have heard that value, quant value t...",Modern Value
1,20203.0,B8467,1,The Economics and Politics of Digital Disruption,You canâ€™t disrupt any industry without dislo...,The Economics Politics Digital Disruption
2,20213.0,B8467,1,The Economics and Politics of Digital Disruption,You can&rsquo;t disrupt any industry without d...,The Economics Politics Digital Disruption
3,20181.0,B9651,1,MS Marketing Analytics,You are a manager in a large corporation taske...,MS Marketing Analytics
4,20163.0,B8617,1,Marketing Research and Analytics,You are a manager in a large corporation taske...,Marketing Research Analytics


In [14]:
print("df count:", df['clean_name'].count())
print("desc count:",desc['clean_name'].count())

df count: 147048
desc count: 14613


In [18]:
df_desc = df.merge(desc, on= "clean_name", how="inner")
df_desc.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name,yearTerm,course_number,section,course_title,course_description
0,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20153.0,B8118,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
1,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20113.0,B9830,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
2,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20123.0,B9830,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
3,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20143.0,B8118,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
4,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20163.0,B8118,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."


In [20]:
df_desc.drop_duplicates()
df_desc.head()

,Unnamed: 0,Person_ID_Number,uni,Grad_Year,Unique_Course_ID,Core_Elective,Course_Identifier,Subject_Name,Credits_Awarded_Count,Final_Grade_Code,...,SIS Citizenship,Future Job Function,Future Job Industry,combined_name,clean_name,yearTerm,course_number,section,course_title,course_description
0,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20153.0,B8118,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
1,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20113.0,B9830,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
2,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20123.0,B9830,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
3,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20143.0,B8118,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."
4,5085,C002941648,aa2301,2016,20153DROMB8118001,Elective,DROMB8118,"DEC,RISK & OPERATIONS MGT",1.5,P1+,...,United States,Fin. - Management,Healthcare / Biotechnology / Pharmaceuticals,Transforming the Delivery of Healthcare Services,Transforming Delivery Healthcare Services,20163.0,B8118,1,Transforming the Delivery of Healthcare Services,"This course provides an advanced, critical ana..."


In [19]:
print("df_desc count:", df_desc['clean_name'].count())

df_desc count: 7202670
